In [1]:
!pip install keras

In [2]:
!pip install tensorflow

In [3]:
!pip install --upgrade keras

In [4]:
!pip install --upgrade tensorflow

In [5]:
import tensorflow as tf

In [6]:
from tensorflow import keras

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#there are total 13 features
#exited is the target column

we have to predict whether customer in the bank will be exiting or not

In [8]:
#importing the dataset
dataset = pd.read_csv("Churn_Modelling.csv")
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [9]:
dataset.shape

(10000, 14)

#### Data preprocessing

In [10]:
#we will be using independent features from CreditScore to Estimated salary 
#Exited will be the target variable

In [11]:
X = dataset.iloc[:,3:13]
y = dataset.iloc[:,13]

In [12]:
#create dummy variables
geography = pd.get_dummies(X['Geography'],drop_first=True)
gender = pd.get_dummies(X['Gender'],drop_first=True)

In [13]:
#concatenate the dataframes
X = pd.concat([X,geography,gender],axis=1)
X.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Germany,Spain,Male
0,619,France,Female,42,2,0.00,1,1,1,101348.88,0,0,0
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0,1,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,0,0,0
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0,0,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0,1,0


In [14]:
#drop unnecessary columns
X = X.drop(['Geography','Gender'],axis=1)
X.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Germany,Spain,Male
0,619,42,2,0.00,1,1,1,101348.88,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,1,0
2,502,42,8,159660.80,3,1,0,113931.57,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,1,0


In [15]:
#splitting the dataset into training and test data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=0)

In [16]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(8000, 11)
(2000, 11)
(8000,)
(2000,)


In [17]:
#Feature scaling
from sklearn.preprocessing import StandardScaler
sc= StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

#### perform Hyper parameter optimization

In [18]:
!pip install keras scikit-learn

In [19]:
# # from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
# from sklearn.model_selection import GridSearchCV
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense
#error - ModuleNotFoundError: No module named 'tensorflow.keras.wrappers'

In [20]:
print(tf.__version__)

2.13.0


In [21]:
# from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
# from tf.keras.wrappers.scikit_learn import KerasClassifier

In [22]:
# solve the above error of module

In [23]:
!pip install tensorflow scikeras scikit-learn

In [24]:
from scikeras.wrappers import KerasClassifier

In [25]:
from sklearn.model_selection import GridSearchCV

In [26]:
from keras.models import Sequential

In [27]:
from keras.layers import Dense, Activation, Embedding, Flatten, LeakyReLU, BatchNormalization

In [28]:
from keras.activations import relu, sigmoid

In [29]:
def create_model(layers, activation):
    model = Sequential()
    for i , nodes in enumerate(layers):
        if i == 0 :
            model.add(Dense(nodes,input_dim = X_train.shape)) # for first layer we have to provide the input dimension
#             model.add(Activation(activation)) #which activation to use
            model.add(Activation('sigmoid')) #which activation to use
            model.add(Dropout(0.3)) #dropout layer
        else:
            model.add(Dense(nodes))
#             model.add(Activation(activation))
            model.add(Activation('sigmoid'))
            model.add(Dropout(0.3))
        model.add(Dense(units = 1, kernel_initializer = 'glorot_uniform', activation = 'sigmoid')) #last layer 
        model.compile(optimizer = 'adam',loss = 'binary_crossentropy', metrics = ['accuracy'])
        return model

        

In [30]:
model = KerasClassifier(build_fn=create_model,verbose = 0)

In [32]:
layers = [[20],[40,20],[40,30,15]] 
# one hidden layer with 20 neurons
#two hidden layers with 40 and 20 neurons respectively
#three hidden layers with 40, 30, 15 neurons respectively

In [33]:
# activations = ['sigmoid', 'relu']

In [37]:
# param_grid = dict(layers = layers, activation = activations, batch_size = [128,356], epochs = [30])
param_grid = dict(layers = layers, batch_size = [128,356], epochs = [30])

In [38]:
grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = 5)

In [39]:
%%time
grid_result = grid.fit(X_train, y_train)
# ValueError: Invalid parameter activation for estimator KerasClassifier.
# This issue can likely be resolved by setting this parameter in the KerasClassifier constructor:
# `KerasClassifier(activation=sigmoid)`
# Check the list of available parameters with `estimator.get_params().keys()`

ValueError: Invalid parameter layers for estimator KerasClassifier.
This issue can likely be resolved by setting this parameter in the KerasClassifier constructor:
`KerasClassifier(layers=[20])`
Check the list of available parameters with `estimator.get_params().keys()`

In [ ]:
#check the errors again and solve the example till end

In [ ]:
#Model best results

In [ ]:
print(grid_result.best_score_,grid_result.best_params_)

In [ ]:
pred_y = grid.predict(X_test)

In [ ]:
y_pred = (pred_y > 0.5)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [ ]:
cm = confusion_matrix(y_pred, y_test)

In [ ]:
score = accuracy_score(y_pred, y_test)